# Proyecto integrador

Miguel Ángel Pérez López A01750145

Ariadna Huesca Coronado A01749161

Lenguaje: GO

https://golang.org/ref/spec#Floating-point_literals

http://www.regular-expressions.info/unicode.html#prop

In [1]:
;; External function required for this notebook.
(require '[clojure.test :refer [is]])

nil

## Pruebas unitarias

### Variable

In [2]:
;;; Regular expression:
(def GO-variable #"(?xi)
                    [\p{L}_][\p{L}\p{N}_]*") 
(is (re-matches GO-variable "a"))
(is (re-matches GO-variable "_x9"))
(is (re-matches GO-variable "ThisVariableIsExported"))
(is (re-matches GO-variable "αβ"))
(is (not (re-matches GO-variable ".")))

true

STRING

In [3]:
(def GO-string #"(?xi)
                    `[^`]*`|
                    \"([^\\\"]|
                        \\ 
                        ([abfnrtv\\\"`]|
                          U[\dA-F]{4}|
                          [0-7]{3}|
                          x[\dA-F]{2}))* 
                    \"") 
(is (re-matches GO-string "`abc`"))
(is (re-matches GO-string "`\n \n`"))
(is (re-matches GO-string "\"\\\"\""))
(is (re-matches GO-string "\"Hello, world!\\n\""))
(is (re-matches GO-string "\"日本語\""))
(is (re-matches GO-string "\"\\u65e5本\\U00008a9e\""))
(is (re-matches GO-string "\"\\xff\\u00FF\""))
(is (not (re-matches GO-string "\"\\0\"")))
(is (not (re-matches GO-string "\"\\xa\"")));este era con U
(is (not (re-matches GO-string "\"\\UabcZ\"")));este era con U
(is (not (re-matches GO-string "\"\\\"")));este era con U

true

In [4]:
(def GO-string #"(?xi)
                    (import|var|true|false)\s") 
(is (re-matches GO-string "import "))
(is (re-matches GO-string "var"))
(is (re-matches GO-string "true"))
(is (re-matches GO-string "false"))
(is (not (re-matches GO-string " ")))
(is (not (re-matches GO-string "importar")))
(is (not (re-matches GO-string "_var")))


FAIL in () (NO_SOURCE_FILE:4)
expected: (re-matches GO-string "var")
  actual: (not (re-matches #"(?xi)
                    (import|var|true|false)\s" "var"))

FAIL in () (NO_SOURCE_FILE:5)
expected: (re-matches GO-string "true")
  actual: (not (re-matches #"(?xi)
                    (import|var|true|false)\s" "true"))

FAIL in () (NO_SOURCE_FILE:6)
expected: (re-matches GO-string "false")
  actual: (not (re-matches #"(?xi)
                    (import|var|true|false)\s" "false"))


true

### Asignación

### Version final hexadecimal

0[xX][_]*([0-9a-fA-F]+(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*|[0-9a-fA-F]*(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+)(?:_[0-9a-fA-F]+)*[pPeE][+-]??[0-9a-fA-F]+(?:_[0-9a-fA-F]+)*

In [5]:
;0[oO][_]*[0-7]+(_[0-7]+)* #octal
;0[xX][0-9a-fA-F]+(_[0-9a-fA-F]+)* #hexadecimal
;0[bB][01]+(_[01]+)* #binario

;[0]+(_\d+)*\d+ #enteros empezando con 0
;[1-9]+\d*(_\d+)* #enteros que no empiezan con 0

;\d*(_\d)*\.\d*([eE][+-]*\d+(_\d)*)* #flotantes con enteros

;#flotantes hexadecimales como 0x2_3.p10_12_2222_2 sin tener numeros despues del .
;0[xX][_]*[0-9a-fA-F]+(_[0-9a-fA-F])*\.p[0-9a-fA-F]+(_[0-9a-fA-F]+)* 
;final casi bueno 0[xX][_]*[0-9a-fA-F]*(_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*(_[0-9a-fA-F]+)*[pP][+-]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*
;caso que acepta 1.p 0[xX][_]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*(_[0-9a-fA-F]+)*[pP][+-]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*
;caso que acepta .1p 0[xX][_]*[0-9a-fA-F]*(_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*[pP][+-]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*

In [6]:
(def mike2 #"(?xi)
 >>={0,1}|<<={0,1} #aritmetico
| [=><!]=| [><] #comparacion
| [-+|^*&/%]??= #asignacion
| |[|]{2}|&{2}|! #logico
| [-+*/%&|^]|(&^) #aritmetico
| (\s)
  ")

#'user/mike2

In [7]:
(def mike #"(?xi)
  0[oO][_]*[0-7]+(?:_[0-7]+)*|0[xX][0-9a-fA-F]+(?:_[0-9a-fA-F]+)*
  |0[bB][01]+(?:_[01]+)*|(?:[0]+\d*(?:_\d+)*
  |[1-9]+\d*(?:_\d+)*)[eE][+-]??(?:[0]+\d*(?:_\d+)*|[1-9]+\d*(?:_\d+)*)
  |[0]+\d*(?:_\d+)*\d+| [1-9]+\d*(?:_\d+)*|0 #enteros
  | \d*(?:_\d)*\.\d*(?:[eE][+-]??\d+(?:_\d)*)*
  |0[xX][_]*([0-9a-fA-F]+(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*
  |[0-9a-fA-F]*(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+)(?:_[0-9a-fA-F]+)*[pPeE][+-]??[0-9a-fA-F]+(?:_[0-9a-fA-F]+)* #hexadecimales flotantes
  | >>={0,1}|<<={0,1}|[-+*/%&|^]|(&^)#aritmetico
  | [=><!]=| [><] #comparacion
  | [-+|^*&/%]??= #asignacion
  | |[|]{2}|&{2}|! #logico
  | (?:// .* )      #Comentario
  | (?: \s )
  ")

#'user/mike

In [8]:
;enteros
(is (re-matches mike "0"))
(is (re-matches mike "3"))
(is (re-matches mike "4_4543_35435_2"))
(is (re-matches mike "342_3242_2342"))
(is (re-matches mike "7_7"))
(is (re-matches mike "00_232_232"))
(is (re-matches mike "0_232_232"))
(is (re-matches mike "8699874238403009"))
(is (re-matches mike "43216400"))
(is (re-matches mike "00089970"))
(is (re-matches mike "08989"))
(is (re-matches mike "7678575"))
(is (re-matches mike "0o6565467"))
(is (re-matches mike "0x435F564A"))
(is (re-matches mike "0X000_3_f565"))
(is (re-matches mike "0x47464456f_32f_3_3"))
(is (re-matches mike "0b10100_01_1"))
(is (re-matches mike "0B00_1_111"))
(is (re-matches mike "0o_6_3432_2_34"))
(is (re-matches mike "0O4_43_2432"))
(is (re-matches mike "0_54654654"))
(is (re-matches mike "//Hello"))
(is (re-matches mike "0xBad_Face"))
;flotantes enteros
(is (re-matches mike "0."))
(is (re-matches mike "72.40"))
(is (re-matches mike "7_2.40"))
(is (re-matches mike "072.40"))
(is (re-matches mike "2.71828"))
(is (re-matches mike "1.e+0"))
(is (re-matches mike "1.e+0_4_4_4"))
(is (re-matches mike "6.67428e-11"))
;checar este caso
(is (re-matches mike ".25"))
(is (re-matches mike ".12345E+5"))
(is (re-matches mike "1_5."))
(is (re-matches mike "0.15e+0_2"))
;flotantes con e sin punto
(is (re-matches mike "1E6"))
(is (re-matches mike "0E6"))
(is (re-matches mike "434_43e423"))
(is (re-matches mike "00565_443_4e4_4"))
(is (not (re-matches mike "4534545e_e")))
;flotantes hexadecimales
(is (re-matches mike "0x2.p10"))
(is (re-matches mike "0x2_3.p10_12_2222_2"))
(is (re-matches mike "0X1.p1"))
(is (re-matches mike "0x_4.p43"))
(is (re-matches mike "0x_4p45"))
(is (re-matches mike "0x1p-2"))
(is (re-matches mike "0x3_3._32p3"))
(is (re-matches mike "0x1.Fp+0"))
(is (re-matches mike "0x.8p-0"))
(is (re-matches mike "0X_1FFFP-16"))
(is (re-matches mike "0x15e-2"))
(is (re-matches mike "0x.1p1"))
;este caso es erróneo
(is (not (re-matches mike "0x.p1")))
;operadores
(is (re-matches mike "+="))
(is (re-matches mike "-="))
(is (re-matches mike "|="))
(is (re-matches mike "=="))
(is (re-matches mike "^="))
(is (re-matches mike "<"))
(is (re-matches mike ">"))
(is (re-matches mike ">>"))
(is (re-matches mike "<<"))
(is (re-matches mike "<<="))
(is (re-matches mike ">>="))
(is (not (re-matches mike ">>>=")))
(is (not (re-matches mike "<<<=")))
(is (re-matches mike "||"))
(is (re-matches mike "&&"))
(is (re-matches mike "!"))

["!" nil nil]

In [9]:
(def mike #"(?xi)
  0[oO][_]*[0-7]+(?:_[0-7]+)*                 #octal
| 0[xX][0-9a-fA-F]+(?:_[0-9a-fA-F]+)*         #hexadecimal
| 0[bB][01]+(?:_[01]+)*                       #binario
| (?:[0]+\d*(?:_\d+)*|[1-9]+\d*(?:_\d+)*)[eE][+-]??(?:[0]+\d*(?:_\d+)*|[1-9]+\d*(?:_\d+)*) #flotantes con E sin punto
| [0]+\d*(?:_\d+)*\d+                         #enteros empezando con 0
| [1-9]+\d*(?:_\d+)*                          #enteros que no empiezan con 0
| \d*(?:_\d)*\.\d*(?:[eE][+-]??\d+(?:_\d)*)*        #flotantes con enteros
| 0[xX][_]*([0-9a-fA-F]+(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*|[0-9a-fA-F]*(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+)(?:_[0-9a-fA-F]+)*[pPeE][+-]??[0-9a-fA-F]+(?:_[0-9a-fA-F]+)* #hexadecimales flotantes
| >>={0,1}|<<={0,1} #aritmetico
| [=><!]=| [><] #comparacion
| [-+|^*&/%]??= #asignacion
| |[|]{2}|&{2}|! #logico
| [-+*/%&|^]|(&^) #aritmetico
| (?:// .* )                                  #Comentario
| ( \s ) ")

#'user/mike

## Tareas

1. Entero 1
2. FLotante 1
3. Comentario 1
4. Variable 1
5. String
6. Operadores (Asignación) = += *= Mmike
7. Lógico (> < ==) Mike
8. Aritmético(+ - *) mike
9. Constantes (True, False, var) Ari
10. Paréntesis y llaves Ari

1. fn2: exportar archivo html
2. fn1: (inicio intermedio final) : string 
3. intermedio: ("<span class=" " resultado(float,var) ">"  contenido "</span>"
4. contenido: (< (menor que)   ➔   &lt; > (mayor que)   ➔   &gt; & (ampersand)   ➔   &amp;)

### Hilos
1. 20 archivos diferentes
2. Script para copiar los archivos - Ari
3. Solucion Secuencial
4. Hilos
5. Estadisticas

### Otros
1. Estilos de clojure - Mike
2. Poster
3. Revision

## Secuencial

### Normal

In [58]:
(def regex #"(?xi)
  (\d*(?:_\d)*\.\d*(?:[eE][+-]??\d+(?:_\d)*)* #Grupo 1 Flotantes
    |0[xX][_]*(?:[0-9a-fA-F]+(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*
    |[0-9a-fA-F]*(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+)
    (?:_[0-9a-fA-F]+)*[pPeE][+-]??[0-9a-fA-F]+(?:_[0-9a-fA-F]+)*)
  | (0[oO][_]*[0-7]+(?:_[0-7]+)*              #Grupo 2 enteros
    |0[xX][0-9a-fA-F]+(?:_[0-9a-fA-F]+)*
    |0[bB][01]+(?:_[01]+)*|(?:[0]+\d*(?:_\d+)*
    |[1-9]+\d*(?:_\d+)*)[eE][+-]??(?:[0]+\d*(?:_\d+)*|[1-9]+\d*(?:_\d+)*)
    |[0]+\d*(?:_\d+)*\d+|[1-9]+\d*(?:_\d+)*|0)
  | ( // .*|[/][*][^*]*[*]+(?:[^*/][^*]*[*]+)*[/]) # Grupo 3: Comentario
  | ((?:import|var|true|false|func|switch|if|return|case|for|new|Float|struct|in|type)[^A-Z \d]*?)                   # Grupo 4: Constantes
  | ('[^']*?'|\"(?:[^\\]|\\(?:[abfnrtv\\\"`]
    |[^\"\n\\]|U[\dA-F]{4}|[0-7]{3}|x[\dA-F]{2}))*?\")   # Grupo 5: String
  | ([-+|*&/%^]??=|<<=|>>=)                   # Grupo 6: Operador de Asignacion
  | ([|]{2}|&{2}|!)                           # Grupo 7: Operador Lógico
  | (>>|<<|(?:&\^)|[-+*/%&|^])                # Grupo 8: Operador Aritmético
  | ( [\p{L}_][\p{L}\p{N}_]* )                # Grupo 9: Variable
  | ( [\(\)\{\}] )                            # Grupo 10: Puntuación
  | ( \s )                                    # Grupo 11: Otro
  | ( . )                                     # Grupo 12: Carácter inválido")

#'user/regex

In [11]:
(defn tokenize-file
  [file-name]
  (->> (re-seq regex (slurp (clojure.string/join ["archivos/" file-name])))
       (map (fn [match]
              (let [token (match 0)]
                (cond
                  (match 1) [token "Flotante"]
                  (match 2) [token "Entero"]
                  (match 3) [token "Comentario"]
                  (match 4) [token "Constante"]
                  (match 5) [token "String"]
                  (match 6) [token "Asignacion"]
                  (match 7) [token "Logico"]
                  (match 8) [token "Aritmetico"]
                  (match 9) [token "Variable"]
                  (match 10) [token "Puntuacion"]
                  (match 11) [token "Espacio"]
                  (match 12) [token "Invalido"]))))))

#'user/tokenize-file

### Prueba local

In [12]:
(defn table-lines
  [file-name]
  (->> (tokenize-file file-name)
       (map (fn [t] (format "%-30s%s" (t 0) (t 1))))))

#'user/table-lines

In [13]:
(defn line
  [character size]
  (clojure.string/join (repeat size character)))

#'user/line

In [14]:
(defn table-string 
  [file-name]
  (clojure.string/join \newline
    (concat [(line \= 50)]
            [(format "%-30s%s" "Token" "Tipo")]
            [(line \= 50)]
            (table-lines file-name)
            [(line \= 50)])))

#'user/table-string

In [15]:
(defn print-token-table
  [file-name]
  (println (table-string file-name)))

#'user/print-token-table

No se porque marca stack over flow unu

In [16]:
;cree una carpeta llamada "pruebaParaleloBorrar" donde puse dos archivos go, pero sólo con 60 líneas cada uno
(tokenize-file "Archivos_prueba/pruebaParaleloBorrar/binary128.go")

Execution error (FileNotFoundException) at java.io.FileInputStream/open0 (FileInputStream.java:-2).
archivos\Archivos_prueba\pruebaParaleloBorrar\binary128.go (El sistema no puede encontrar la ruta especificada)


class java.io.FileNotFoundException: 

### En HTML

In [17]:
(defn reemplaza [string]
  "Remplaza carácteres &,<,> por sus equivalentes en html."
  (->> (clojure.string/replace string #"&" "&amp")
       (map (fn [string] (clojure.string/replace string #"<" "&lt")))
       (map (fn [string] (clojure.string/replace string #">" "&gt")))
       (reduce str)))

#'user/reemplaza

In [18]:
(defn etiquetas [file-name]
    "Genera las etiquetas HTML."
  (->> (tokenize-file file-name)
    ; Si (t 1) == Espacio dejarlo normal, si no el span
       (map (fn [t] (if (= (t 1) "Espacio")
                      (t 0)
                      (clojure.string/join 
                       ["<span id = '" 
                        (t 1) 
                        "'>" 
                        (reemplaza (t 0)) 
                        "</span>"]))))
       (reduce str)))

#'user/etiquetas

In [19]:
(defn final [file-name]
  (clojure.string/join [(slurp "inicio.txt")
                        (etiquetas file-name)
                        (slurp "final.txt")]))

#'user/final

In [20]:
;funcion final
(defn procesa-texto [file-name]
  (spit (clojure.string/join ["Resultado/" file-name ".html"]) 
        (final (clojure.string/join [file-name ".go"]))))

#'user/procesa-texto

In [21]:
(defn get-file-info
  [dir]
  (map (fn [f]
         [(.getPath f) (.isFile f) (.getName f)]);obtiene 3 informaciones de cada archivo
       (file-seq (clojure.java.io/file dir))));   para usarlo en la función valid-file-names qu esta abajo

#'user/get-file-info

In [22]:
;Esta función sirve para obtener todos los nombres que se encuentren en la carpeta a analizar.
;Como en la función de procesa-texto se pide el nombre del archivo sin extensión,
;cree esta función
(defn valid-file-names
  [dir]                                        ;obtiene los nombres del directorio
  (->> (for [f (get-file-info dir) :when (f 1)];(f 1) es true o false si es archivo o no
            (f 2)) ;si (f 1) es verdadero se incluye el nombre del archivo
       (remove (fn [string1] (re-seq #"(?xi) (checkpoint|[.]css|[.]html)" string1)));quita archivos checkpoint que hace jupyterlab, archivos html y css
       (map (fn [string] (clojure.string/replace string #".go" "")));los archivos go le quita su extensión
       (map (fn [name1] (clojure.string/join [dir "/" name1])))));le agrega su extensión que se pone como parámetro dir
                                                    ;porque si sólo se queda el nombre, no va a encontrar el archivo en caso de que
                                                    ;el archivo este en otro directorio

#'user/valid-file-names

In [56]:
(valid-file-names "Archivos_prueba/10archivos")

("Archivos_prueba/10archivos/Untitled.ipynb")

## La función de abajo es la que hace todo la parte 1

Esta es la parte que creo que tiene que ir con loop recur y que se utilizará en el pmap de la función paralela

In [24]:
;(defn part-one     ;esta función recibe el directorio y aplica la función procesa-texto
;  [directory]
;  (map (fn [string] (procesa-texto string)) (valid-file-names directory)))

(defn part-one2    ;esta función hace lo mismo pero recibe una lista de los nombres
  [directory]
  (map procesa-texto directory))

#'user/part-one2

In [51]:
(def archivos-prueba (map #(clojure.string/replace % #"archivos/" "") (valid-file-names "archivos")))
(println archivos-prueba)

(binary128 binary16_test binary_as_string_codec)


nil

In [57]:
(part-one2 archivos-prueba)

(nil nil nil)

## Paralelo

In [28]:
;esta función hace que haga una secuencia 
(defn ranges-str
  [p strings]
  (partition p strings))

#'user/ranges-str

In [29]:
(println archivos-prueba)

(binary_as_string_codec)


nil

In [30]:
(partition 2 archivos-prueba)

()

In [31]:
;esta es la función paralela de la parte 2
(defn parallel
  [p directory]
  (->> (ranges-str p (valid-file-names directory));busca los nombres en el directorio y regresa una secuencia particionada
       (pmap (fn [lst-str] (procesa-texto lst-str)))));procesa la secuencia particionada con un pmap

;esta función es lo mismo que la de arriba, nada más que la puse para que reciba una lista con los nombres de los archivos
;porque hice otra función? porque la función de arriba no jala unu,falla al querer procesar lo que regresa la línea 4
;regresa algo asi: (("binary128" "binary16_test") ("esto" "es") ("una" "prueba")) y el pmap no sabe que hacer con
;una secuencia de secuencias uwu
(defn parallel-prueba
  [string]
  (pmap (fn [n] (procesa-texto n)) string))

(defn parallel-Ariadna  
  [p archivo]
  (->> (partition p archivo)
       (pmap 
         #(part-one2 %))))

#'user/parallel-Ariadna

In [32]:
(parallel-Ariadna 2 archivos-prueba)

()

## Pruebas

In [33]:
(doseq [i (range 5)]
  (time (part-one2 
         archivos-prueba)))

"Elapsed time: 0.0155 msecs"
"Elapsed time: 0.0026 msecs"
"Elapsed time: 0.0014 msecs"
"Elapsed time: 0.0015 msecs"
"Elapsed time: 0.0016 msecs"


nil

In [34]:
(doseq [i (range 5)]
  (time (parallel-Ariadna
         2 archivos-prueba)))

"Elapsed time: 0.0661 msecs"
"Elapsed time: 0.0304 msecs"
"Elapsed time: 0.0203 msecs"
"Elapsed time: 0.018 msecs"
"Elapsed time: 0.0172 msecs"


nil